In [1]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

# Paths to saved data files
output_dir = "../../../Datasets/12-lead electrocardiogram database/FirstRevFull5000Sample"
train_data_file = os.path.join(output_dir, "X_train.txt")
train_labels_file = os.path.join(output_dir, "y_train.txt")
test_data_file = os.path.join(output_dir, "X_test.txt")
test_labels_file = os.path.join(output_dir, "y_test.txt")

# Load data and maintain 3D shape
def load_split(data_file, labels_file, sample_shape=(5000, 12)):
    data = []
    labels = []

    with open(data_file, "r") as f_data:
        for line in f_data:
            sample = np.array(line.strip().split(), dtype=np.float32)
            sample = sample.reshape(sample_shape)  # Reshape to (5000, 12)
            data.append(sample)

    with open(labels_file, "r") as f_labels:
        labels = [line.strip() for line in f_labels]

    return np.array(data), np.array(labels)

# Load train and test data in 3D format
X_train, y_train = load_split(train_data_file, train_labels_file)
X_test, y_test = load_split(test_data_file, test_labels_file)

# Scale each feature across all samples independently
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Visualize some samples to check
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# Encode labels using one-hot encoding
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# Define the CNN-LSTM model for time-series analysis
input_shape = X_train.shape[1:]  # (5000, 12)
num_classes = y_train.shape[1]

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
y_pred = model.predict(X_test)
y_pred_labels = lb.inverse_transform(y_pred)
y_true_labels = lb.inverse_transform(y_test)
print("\nClassification Report:")
print(classification_report(y_true_labels, y_pred_labels))

2024-11-08 20:32:51.319805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 20:32:51.330838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 20:32:51.334207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 20:32:51.343210: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 20:32:51.912260: W tensorflow/compiler/tf2

Train data shape: (7984, 5000, 12)
Test data shape: (2662, 5000, 12)


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1731076512.978662  228929 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731076513.012965  228929 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:

Epoch 1/10


2024-11-08 20:35:16.371877: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1731076516.421348  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.450043  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.450665  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.460391  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.461796  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.463075  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.467075  230458 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076516.467648  230458 gpu_t

...

W0000 00:00:1731076525.969752  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.970393  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.970966  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.971566  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.972203  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.972800  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.973393  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.973991  230456 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076525.974581  230456 gp

200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.0444 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.0411 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.0420 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.0397 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.0423 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.0476 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.0433 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.0395 - loss: nan - val_accuracy: 0.0401 - val_loss: nan
Epoch 10/10
200/200 ━━━━━━━

W0000 00:00:1731076597.646819  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.648374  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.648890  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.649408  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.649924  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.650441  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.650963  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.651478  230452 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731076597.651993  230452 gp

84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step

Classification Report:
              precision    recall  f1-score   support

          AF       0.04      1.00      0.08       111
        AFIB       0.00      0.00      0.00       445
          AT       0.00      0.00      0.00        30
       AVNRT       0.00      0.00      0.00         4
        AVRT       0.00      0.00      0.00         2
       SAAWR       0.00      0.00      0.00         2
          SB       0.00      0.00      0.00       972
          SI       0.00      0.00      0.00       100
          SR       0.00      0.00      0.00       457
          ST       0.00      0.00      0.00       392
         SVT       0.00      0.00      0.00       147

    accuracy                           0.04      2662
   macro avg       0.00      0.09      0.01      2662
weighted avg       0.00      0.04      0.00      2662



/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control